<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/applications/mangrove_class/08_Mangrove_Train_Indices_Class_as_Asset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The file path on google drive for ROI output vector file.
vec_cls_roi_file = "/content/drive/MyDrive/mangrove_chng_cls/roi_poly.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0

In [ ]:
# Specify the paths to the vector files with the training points for each of
# classes. Note these files include samples for a larger areas than defined
# in the roi file.
vec_mng_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/mng_smpls.geojson'
vec_wtr_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/wtr_smpls.geojson'
vec_oth_smpls_file = '/content/drive/MyDrive/mangrove_chng_cls/oth_smpls.geojson'

In [ ]:
# The following function calls convert the training sample points to GEE
# point geometries. In addition, the points a clipped to the ROI for the
# region being classified and subsampled to reduce the number of points
# used to train the classifier as this reduced the memory footprint of
# the notebook which is limited.
gee_mng_pts = pb_gee_tools.convert_types.get_gee_pts(vec_mng_smpls_file, rnd_smpl = 7500, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)
gee_wtr_pts = pb_gee_tools.convert_types.get_gee_pts(vec_wtr_smpls_file, rnd_smpl = 7500, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)
gee_oth_pts = pb_gee_tools.convert_types.get_gee_pts(vec_oth_smpls_file, rnd_smpl = 7500, rnd_seed = 42, vec_roi_file=vec_cls_roi_file)


In [ ]:
# Merge the training samples into a single Feature Collection with a variable
# for the ID of each class.
train_smpls = ee.FeatureCollection([
    ee.Feature(gee_mng_pts, {'class': 1}),
    ee.Feature(gee_wtr_pts, {'class': 2}),
    ee.Feature(gee_oth_pts, {'class': 3}),
])

In [ ]:
# Get the vector polygon from the vector file as a Google Earth Polygon
roi_gee_poly = pb_gee_tools.convert_types.convert_vector_to_gee_polygon(vec_cls_roi_file)

In [ ]:
# Load the Sentinel-2 imagery.
s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
    aoi=roi_gee_poly,
    start_date=start_date,
    end_date=end_date,
    cloud_thres = 70,
)

In [ ]:
# A function which will be used to calculate a number of image band indices
# for each input image.
def calc_band_indices(img):
    img = img.multiply(.0001).float()
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = img.normalizedDifference(["B8", "B11"]).rename("NDWI")
    nbr = img.normalizedDifference(["B8", "B12"]).rename("NBR")
    evi = img.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
      {
          'NIR': img.select('B8'),
          'RED': img.select('B4'),
          'BLUE': img.select('B2'),
      },
    ).rename(['EVI'])
    mvi = img.expression(
      '((NIR - GREEN) / (SWIR - GREEN))',
      {
          'NIR': img.select('B8'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['MVI'])
    remi = img.expression(
      '((REDEDGE - RED) / (SWIR + GREEN))',
      {
          'REDEDGE': img.select('B6'),
          'RED': img.select('B4'),
          'SWIR': img.select('B11'),
          'GREEN': img.select('B3'),
      },
    ).rename(['REMI'])
    return img.addBands([ndvi, ndwi, nbr, evi, mvi, remi])

In [ ]:
# Map the input images to calculate the indices for
# each the input images.
s2_indices_img_col = s2_img_col.map(calc_band_indices)


In [ ]:
# Get the list of image bands.
img_bands = s2_indices_img_col.first().bandNames()
img_bands

In [ ]:
# Sample training data from each of the input images
def sample_img_training(img):
    train_smpl_data = img.sampleRegions(
        collection=train_smpls, properties=["class"], scale=10
    )
    return train_smpl_data

training_data = s2_indices_img_col.map(sample_img_training)
training_data = training_data.flatten()

In [ ]:
# Train the classifier.
rf_cls_mdl = ee.Classifier.smileRandomForest(numberOfTrees=100).train(training_data, "class", img_bands)

In [ ]:
# Save the classifier as an asset in your Google Earth Engine account
asset_id = f'projects/{ee_prj_name}/assets/mng_indices_rf_cls'
task = ee.batch.Export.classifier.toAsset(
  classifier=rf_cls_mdl,
  description=f'mng_test_rf_cls',
  assetId=asset_id
)
task.start()